In [378]:
import pandas as pd
import requests
import numpy as np
from math import radians, cos, sin, asin, sqrt


In [2]:
df = pd.read_csv("supplydata.csv")

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
#Stripping Details to get only the origin and delivery Dates
df["detail"] =  df.details.str[:40] + df.details.str[-65:] 


In [84]:
#Some cleaning
df["detail2"] = df.detail.str.replace(',', '').str.replace('"','').str.replace(']','').str.replace("'","")

In [132]:
#Extracting Dates Function
import datefinder
from datetime import datetime

def cleaner(string):
    try:
        matches = datefinder.find_dates(string)
        return (matches.next() - matches.next()).days
    except StopIteration:
        return -999

In [103]:
df["Time taken"] = df["detail2"].map(cleaner)

In [105]:
df.drop('details', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)


In [107]:
df.to_csv("timetaken.csv")

In [108]:
df = df.drop(df[df['Time taken']<0].index)


In [110]:
df.describe()

,Unnamed: 0,pieces,Time taken
count,428370.000000,428370.000000,428370.000000
mean,253077.997610,1.029376,2.156057
std,145115.073481,0.552665,4.075782
min,1.000000,1.000000,0.000000
25%,127348.250000,1.000000,1.000000
50%,252465.500000,1.000000,1.000000
75%,380047.500000,1.000000,2.000000
max,499598.000000,69.000000,551.000000


In [115]:
df = df.drop(df[df['Time taken']>15].index)


In [118]:
df.describe()

,pieces,Time taken
count,424989.000000,424989.000000
mean,1.028956,1.939156
std,0.547880,1.810264
min,1.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,2.000000
max,69.000000,15.000000


In [170]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('details', axis=1, inplace=True)
df.drop('pieces', axis=1, inplace=True)


In [149]:
df.to_csv("FinalTimeTaken.csv")

In [167]:
df = pd.read_csv("FinalTimeTaken.csv")

In [174]:
df.destinationbranch = df.destinationbranch.str.replace('SDC', '').str.strip()
df.originbranch = df.originbranch.str.replace('SDC', '').str.strip()

In [235]:
df.destinationbranch = df.destinationbranch.str.replace('MAHIPALPUR MDC', 'MAHIPALPUR')
df.originbranch = df.originbranch.str.replace('MAHIPALPUR MDC', 'MAHIPALPUR')

In [236]:
df.head()

,destinationbranch,originbranch,producttype,Time taken
0,BAREILLY,OKHLA,T2,8
1,BAREILLY,OKHLA,T2,4
2,NOIDA,OKHLA,T2,1
3,BAREILLY,OKHLA,T2,4
4,MORADABAD,OKHLA,T2,6


In [237]:
df.destinationbranch.unique()

array(['BAREILLY', 'NOIDA', 'MORADABAD', 'GHAZIABAD', 'MAHIPALPUR',
       'MEERUT', 'KIRTI NAGAR', 'LAXMI NAGAR', 'RANI JHANSI', 'KANPUR',
       'LUCKNOW', 'AMRITSAR', 'JALANDHAR', 'LUDHIANA', 'VARANASI',
       'GURGAON', 'AJMER', 'INDORE', 'WAZIRPUR', 'FARIDABAD', 'CHANDIGARH',
       'JAIPUR', 'GURGAON  EAST', 'OKHLA', 'AGRA', 'AMBALA', 'ALLAHABAD',
       'HARIDWAR', 'PATIALA', 'BHOPAL', 'KOTA', 'JODHPUR', 'PANIPAT',
       'SONIPAT', 'VISHNU GARDEN', 'DEHRADUN', 'KARNAL', 'ANDHERI EAST',
       'YELAHANKA', 'PANCHSHEEL VIHAR', 'BHIWANDI', 'AZADPUR'], dtype=object)

In [238]:
df.originbranch.unique()

array(['OKHLA', 'WAZIRPUR', 'NOIDA', 'RANI JHANSI', 'CHANDIGARH',
       'LAXMI NAGAR', 'MAHIPALPUR', 'GHAZIABAD', 'ANDHERI EAST', 'GURGAON',
       'VISHNU GARDEN', 'KIRTI NAGAR', 'GURGAON  EAST', 'KANPUR',
       'ALLAHABAD', 'BAREILLY', 'VARANASI', 'JALANDHAR', 'AMRITSAR',
       'FARIDABAD', 'JAIPUR', 'LUDHIANA', 'INDORE', 'BHOPAL', 'MEERUT',
       'KARNAL', 'PANIPAT', 'PATIALA', 'MORADABAD', 'AMBALA', 'YELAHANKA',
       'AGRA', 'LUCKNOW', 'SONIPAT', 'KOTA', 'HARIDWAR', 'DEHRADUN',
       'AJMER', 'JODHPUR', 'PANCHSHEEL VIHAR', 'BHIWANDI'], dtype=object)

In [239]:
list(set(df.destinationbranch.unique()) - set(df.originbranch.unique() ))

['AZADPUR']

In [186]:
list_of_cities = df.originbranch.unique().tolist() + list(set(df.destinationbranch.unique()) - set(df.originbranch.unique() ))

In [217]:
d = {}
for i in list_of_cities:
    d[i] = 1

In [ ]:
#Getting longitudes and latitudes
for key in d:
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=' + key)
    resp_json_payload = response.json()
    d[key] = (resp_json_payload['results'][0]['geometry']['location'])
    time.sleep(0.2)

In [290]:
d

{'AGRA': {u'lat': 27.1766701, u'lng': 78.00807449999999},
 'AJMER': {u'lat': 26.4498954, u'lng': 74.6399163},
 'ALLAHABAD': {u'lat': 25.4358011, u'lng': 81.846311},
 'AMBALA': {u'lat': 30.3781788, u'lng': 76.7766974},
 'AMRITSAR': {u'lat': 31.6339793, u'lng': 74.8722642},
 'ANDHERI EAST': {u'lat': 19.1154908, u'lng': 72.8726952},
 'AZADPUR': {u'lat': 28.7102951, u'lng': 77.1752448},
 'BAREILLY': {u'lat': 28.3670355, u'lng': 79.4304381},
 'BHIWANDI': {u'lat': 19.2812547, u'lng': 73.0482912},
 'BHOPAL': {u'lat': 23.2599333, u'lng': 77.412615},
 'CHANDIGARH': {u'lat': 30.7333148, u'lng': 76.7794179},
 'DEHRADUN': {u'lat': 30.3164945, u'lng': 78.03219179999999},
 'FARIDABAD': {u'lat': 28.4089123, u'lng': 77.3177894},
 'GHAZIABAD': {u'lat': 28.6691565, u'lng': 77.45375779999999},
 'GURGAON': {u'lat': 28.4594965, u'lng': 77.0266383},
 'GURGAON  EAST': {u'lat': 28.4594965, u'lng': 77.0266383},
 'HARIDWAR': {u'lat': 29.9456906, u'lng': 78.16424780000001},
 'INDORE': {u'lat': 22.7195687, u'lng'

In [289]:
del d["MAHIPALPUR MDC"]

In [248]:
coord_dict = d

In [316]:
d.values()

[{u'lat': 13.1004849, u'lng': 77.5940134},
 {u'lat': 30.900965, u'lng': 75.8572758},
 {u'lat': 26.8466937, u'lng': 80.94616599999999},
 {u'lat': 29.6856929, u'lng': 76.9904825},
 {u'lat': 28.8386481, u'lng': 78.7733286},
 {u'lat': 31.3260152, u'lng': 75.57618289999999},
 {u'lat': 28.6868196, u'lng': 77.4509217},
 {u'lat': 28.6504091, u'lng': 77.14442679999999},
 {u'lat': 28.56028, u'lng': 77.2913315},
 {u'lat': 26.2389469, u'lng': 73.02430939999999},
 {u'lat': 28.5355161, u'lng': 77.3910265},
 {u'lat': 25.3176452, u'lng': 82.9739144},
 {u'lat': 25.2138156, u'lng': 75.8647527},
 {u'lat': 28.6691565, u'lng': 77.45375779999999},
 {u'lat': 28.4594965, u'lng': 77.0266383},
 {u'lat': 28.6519345, u'lng': 77.0940718},
 {u'lat': 28.4089123, u'lng': 77.3177894},
 {u'lat': 28.7002168, u'lng': 77.1637899},
 {u'lat': 31.6339793, u'lng': 74.8722642},
 {u'lat': 25.4358011, u'lng': 81.846311},
 {u'lat': 29.07, u'lng': 80.67},
 {u'lat': 28.3670355, u'lng': 79.4304381},
 {u'lat': 22.7195687, u'lng': 75.

In [344]:
df["lon1"] = df["originbranch"].apply(lambda x: d.get(x).get("lng"))
df["lat1"] = df["originbranch"].apply(lambda x: d.get(x).get("lat"))
df["lon2"] = df["destinationbranch"].apply(lambda x: d.get(x).get("lng"))
df["lat2"] = df["destinationbranch"].apply(lambda x: d.get(x).get("lat"))

In [346]:
df.head()

,destinationbranch,originbranch,producttype,Time taken,lon1,lat1,lon2,lat2
0,BAREILLY,OKHLA,T2,8,77.291331,28.56028,79.430438,28.367036
1,BAREILLY,OKHLA,T2,4,77.291331,28.56028,79.430438,28.367036
2,NOIDA,OKHLA,T2,1,77.291331,28.56028,77.391026,28.535516
3,BAREILLY,OKHLA,T2,4,77.291331,28.56028,79.430438,28.367036
4,MORADABAD,OKHLA,T2,6,77.291331,28.56028,78.773329,28.838648


In [385]:
#Haversine Formula to calculate the distance
import math

def distance(lon1, lat1, lon2, lat2):
    radius = 6371

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [387]:
#Vectorize the function and apply 
df["Distance"] = np.vectorize(distance)(df['lon1'], df['lat1'],df['lon2'],df['lat2'])

In [388]:
df.head()

,destinationbranch,originbranch,producttype,Time taken,lon1,lat1,lon2,lat2,Distance
0,BAREILLY,OKHLA,T2,8,77.291331,28.56028,79.430438,28.367036,210.203675
1,BAREILLY,OKHLA,T2,4,77.291331,28.56028,79.430438,28.367036,210.203675
2,NOIDA,OKHLA,T2,1,77.291331,28.56028,77.391026,28.535516,10.119614
3,BAREILLY,OKHLA,T2,4,77.291331,28.56028,79.430438,28.367036,210.203675
4,MORADABAD,OKHLA,T2,6,77.291331,28.56028,78.773329,28.838648,147.821979


In [393]:
df.drop('destinationbranch', axis=1, inplace=True)
df.drop('originbranch', axis=1, inplace=True)
df.drop('lon1', axis=1, inplace=True)
df.drop('lon2', axis=1, inplace=True)
df.drop('lat1', axis=1, inplace=True)
df.drop('lat2', axis=1, inplace=True)


In [401]:
#One hot encoding
df = pd.get_dummies(df)

In [404]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424989 entries, 0 to 424988
Data columns (total 11 columns):
Time taken            424989 non-null int64
Distance              424989 non-null float64
producttype_T1        424989 non-null float64
producttype_T148      424989 non-null float64
producttype_T172      424989 non-null float64
producttype_T196      424989 non-null float64
producttype_T2        424989 non-null float64
producttype_T21400    424989 non-null float64
producttype_T3        424989 non-null float64
producttype_T4        424989 non-null float64
producttype_null      424989 non-null float64
dtypes: float64(10), int64(1)
memory usage: 35.7 MB


In [407]:
df.to_csv("FinalDataSet")

In [409]:
df.dtypes

Time taken            int64  
Distance              float64
producttype_T1        float64
producttype_T148      float64
producttype_T172      float64
producttype_T196      float64
producttype_T2        float64
producttype_T21400    float64
producttype_T3        float64
producttype_T4        float64
producttype_null      float64
dtype: object